## KRX-Bench 합성 데이터셋 생성 with web text 튜토리얼

- **litellm**은 다양한 LLM API를 OpenAI API로 통합하여 사용할 수 있는 라이브러리입니다.
- 본 튜토리얼에서는 OpenAI의 `gpt-4o-mini-2024-07-18` 모델을 활용하여 웹에서 수집된 금융 관련 고품질 텍스트 데이터셋 `amphora/rewrite-se-quant`을 정규화한 뒤, QA Instruction 데이터셋으로 변환하는 방법에 대해 다룹니다.
- 데이터 생성 파이프라인:
 1. 텍스트 데이터를 입력으로 해서 질문 생성
 2. 생성된 질문 세트에 대한 답변 생성

## 1. litellm 설치 및 환경 설정
- 필요 라이브러리: pandas, datasets, random, litellm, os

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
import random
from litellm import completion, batch_completion
import os
import litellm

# OpenAI API key 선언
# os.environ["OPENAI_API_KEY"] = 

## 2. 데이터셋 로드 및 전처리

- `amphora/rewrite-se-quant` 데이터셋은 금융 관련 고품질 웹 텍스트 데이터셋입니다.

In [14]:
# 데이터셋 로드
ds = load_dataset('amphora/rewrite-se-quant')['train']

In [15]:
ds

Dataset({
    features: ['link', 'query', 'output'],
    num_rows: 21950
})

In [16]:
ds['query'][:10]

["I want to know how to price an American call option for non-dividend stock? (with concrete and simple binomial pricing model, with risk neutral assumption)\nI understand that for an European call option, (in Binomial pricing model), the price is simply:\n$$V_n(\\omega) = \\frac{1}{1+r} (PV_{n+1}(\\omega H) + QV_{n+1}(\\omega T) )\\tag1$$\nand from Early execise of American Call on Non-Dividend paying stock. and many other materials also state that early exercise of American call options is not optimal compared with selling it.\nHowever, many materials also state or imply that European calls and American calls are identical when the underlying stock pays no dividends, which means (1) should apply right? (for anytime before maturity)\nThis confuses me as I thought the pricing of an American call option should be:\n$$V_n(\\omega) = max(S(\\omega) - K, \\frac{1}{1+r} (PV_{n+1}(\\omega H) + QV_{n+1}(\\omega T))) \\tag2$$\nSo let's say if I have a deep in-the-money American call option for

## 3. 합성 데이터셋 생성

합성 데이터셋 생성 파이프라인은 다음과 같습니다.
1. 온라인 데이터를 사용해서 질문 데이터셋을 생성합니다.
2. 생성된 질문 데이터셋에 대한 답변을 생성합니다.

In [17]:
qrys = []
for t in ds['query']:
    messages = [
    {"content": "Your job is creating quantitative finance questions in fluent Korean. You will be given a English QF question collected from the web. Restructure it to a test-like question, in formal Korean language. Return the question only.", "role": "system"},
    {"content": t, "role": "user"}]
    qrys.append(messages)

In [18]:
len(qrys)

21950

In [19]:




import time
import openai
import litellm
from tqdm import tqdm 
# messages 생성 및 토큰 계산
responses = []
batch_size = 4  # 한번에 요청할 batch 크기
sleep_interval = 1  # 각 배치 사이의 슬립 시간 (초)
count = 0
# litellm의 verbose 설정
litellm.set_verbose = False  # 초기 설정은 False로

for i in tqdm(range(0, len(qrys[5000:]), batch_size)):
    batch_messages = qrys[i:i + batch_size]

    batch_responses = batch_completion(
        model="gpt-4o-mini-2024-07-18",
        messages=batch_messages
    )
    if 'RateLimitError' in str(batch_responses):
        break
    else :
        responses.extend(batch_responses)
        count += 1

    # "Give Feedback / Get Help" 메시지 확인


    time.sleep(sleep_interval)


  0%|          | 2/4238 [00:13<7:44:49,  6.58s/it]


KeyboardInterrupt: 

In [ ]:
question_resps = [i.choices[0].message.content for i in responses]
total_prompt_tokens_for_q = sum([r.usage.prompt_tokens for r in responses])
total_completion_tokens_for_q = sum([r.usage.completion_tokens for r in responses])

In [9]:
# 답변 생성용 prompt 포맷팅
qrys = []
for t in question_resps:
    messages = [
    {"content":"You are a skilled financial expert in Korea. Make a response for the question. DO NOT introduce yourself.","role":"system"},
    { "content": t,"role": "user"}]
    qrys.append(messages)

# 2. 생성된 질문에 대한 답변 생성



# messages 생성 및 토큰 계산
responses = []
batch_size = 4  # 한번에 요청할 batch 크기
sleep_interval = 1  # 각 배치 사이의 슬립 시간 (초)

# litellm의 verbose 설정
litellm.set_verbose = False  # 초기 설정은 False로






for i in tqdm(range(0, len(qrys), batch_size)):
    batch_messages = qrys[i:i + batch_size]

    batch_responses = batch_completion(
        model="gpt-4o-mini-2024-07-18",
        messages=batch_messages
    )
    if 'RateLimitError' in str(batch_responses):
        break
    else :
        responses.extend(batch_responses)
        count += 1

    # "Give Feedback / Get Help" 메시지 확인


    time.sleep(sleep_interval)

response_resps = [i.choices[0].message.content for i in responses]
total_prompt_tokens_for_a = sum([r.usage.prompt_tokens for r in responses])
total_completion_tokens_for_a = sum([r.usage.completion_tokens for r in responses])

  0%|          | 0/1 [00:20<?, ?it/s]


KeyboardInterrupt: 

In [10]:
qrys

[[{'content': 'You are a skilled financial expert in Korea. Make a response for the question. DO NOT introduce yourself.',
   'role': 'system'},
  {'content': '비배당 주식에 대한 미국식 콜 옵션의 가격 책정 방법에 대해 설명하시오. 구체적이고 간단한 이항 모형을 사용하고 위험 중립 가정을 고려하여 다음 질문에 답하십시오. \n\n유럽식 콜 옵션의 경우, 이항 가격 모형에서 가격은 다음과 같이 표현됩니다:\n$$V_n(\\omega) = \\frac{1}{1+r} (PV_{n+1}(\\omega H) + QV_{n+1}(\\omega T) )\\tag1$$\n\n그러나 미국식 콜 옵션의 조기 행사에 대해 다양한 자료에서 조기 행사가 최적이지 않다고 언급하고 있습니다. 또한, 비배당 주식의 경우 유럽식 콜 옵션과 미국식 콜 옵션이 동일하다는 설명도 존재합니다. \n\n이러한 내용을 바탕으로, 비배당 주식에 대해 깊게 인더머니 상태에 있는 미국식 콜 옵션이 만료되지 않은 경우, 이 옵션의 가격을 찾아보세요. 조기 행사가 최적이 아니기 때문에 미국식 콜 옵션의 가격은 시간 가치를 추가하지 않으며, 따라서 (2) 식을 사용하는 이유는 무엇인지 설명하시오. \n귀하의 결론을 도출하는 데 있어 (1) 식과 (2) 식의 차이를 비교하십시오.',
   'role': 'user'}],
 [{'content': 'You are a skilled financial expert in Korea. Make a response for the question. DO NOT introduce yourself.',
   'role': 'system'},
  {'content': 'GBP 수익률 곡선을 USD OIS 금리 곡선 및 FX 포워드 rates를 사용하여 Quantlib로 계산하려고 합니다. 다른 라이브러리의 출력을 재현하려고 하였으나, 정확하게 일치하지 않는

## 5. 데이터셋 저장 및 확인

In [13]:
len(question_resps),len(response_resps)

(4000, 4000)

In [ ]:
df = pd.DataFrame({'': ds[1000:5000]['query'], 'question': question_resps, 'response': response_resps})

# CSV 파일 저장
df.to_csv("rewrite-se-quant_final.csv")



# HuggingFace Hub 업로드 - token에 개인 HuggingFace 토큰을 입력해주시면 됩니다.
# result_df = Dataset.from_pandas(df)


# df.head()

## 참고자료

- [alvanlii/finance-textbooks](https://huggingface.co/datasets/alvanlii/finance-textbooks)
- [litellm Docs](https://docs.litellm.ai/)
- [Cosmopedia GitHub](https://github.com/huggingface/cosmopedia)
- [Cosmopedia Blog](https://huggingface.co/blog/cosmopedia)
- [Textbooks Are All You Need](https://arxiv.org/pdf/2306.11644)
- [Learning to Generate Instruction Tuning Datasets for Zero-Shot Task Adaptation](https://arxiv.org/abs/2402.18334)